In [2]:
import mailbox
import re
import urllib.parse
import stix2
from bs4 import BeautifulSoup
import email
import requests
import ipaddress
import json


In [13]:
# Load the mbox file
mbox_files = ['emails-enron-legal-mails.mbox', 'phishing3.mbox']

# concatenate the mbox files into one
with open('merged_emails.mbox', 'wb') as outfile:
  for mbox_file in mbox_files:
    with open(mbox_file, 'rb') as infile:
      outfile.write(infile.read())

#view the mbox_file
mbox_file = "merged_emails.mbox"
mbox = mailbox.mbox(mbox_file)


In [15]:
mbox[6].walk()


<generator object walk at 0x0000018B458FDCB0>

In [20]:

# Get the headers
headers = mbox[1].items()
body = mbox[1].get_payload()

# 'Return-Path','Delivered-To', 'Received', 'Reply-To', 'From',

print(headers,body)

[('Message-ID', '<24786206.1075857703103.JavaMail.evans@thyme>'), ('Date', 'Thu, 28 Dec 2000 17:19:00 -0800 (PST)'), ('From', 'phillip.allen@enron.com'), ('To', 'john.lavorato@enron.com'), ('Subject', ''), ('Mime-Version', '1.0'), ('Content-Type', 'text/plain; charset=us-ascii'), ('Content-Transfer-Encoding', '7bit'), ('X-From', 'Phillip K Allen'), ('X-To', 'Lavorato, John </o=ENRON/ou=NA/cn=Recipients/cn=Jlavora>'), ('X-cc', ''), ('X-bcc', ''), ('X-Folder', '\\jlavora\\COMP'), ('X-Origin', 'Lavorado-J'), ('X-FileName', 'jlavora.pst')] Trading Profits

P. Allen		200
M. Grigsby	463
Rest of Desk	282

Total		945



I view my bonus as partly attributable to my own trading and partly to the group's performance.  Here are my thoughts.



		Minimum		Market			Maximum

Cash		2 MM			4 MM			6 MM
Equity		2 MM			4 MM			6 MM


Here are Mike's numbers.  I have not made any adjustments to them.


		Minimum		Market			Maximum

Cash		2 MM			3 MM			4 MM
Equity		4 MM			7 MM			12 MM


I have given him an "e

In [21]:
# define regular expressions for IP addresses, email addresses, and URLs
ip_pattern = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
email_pattern = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
url_pattern = re.compile(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')


In [22]:
import time
import requests
import urllib.parse

url_list = []

def download_url_list():
    global url_list, url_list_timestamp
    try:
        with requests.get('https://urlhaus.abuse.ch/downloads/text/') as response:
            url_list = response.text.splitlines()
            url_list_timestamp = time.time()
    except Exception as e:
        print(f"Error downloading URL list: {str(e)}")
        url_list = []
        url_list_timestamp = 0

download_url_list()



In [23]:
url_list

['################################################################',
 '# abuse.ch URLhaus Plain-Text URL List (URLs only)             #',
 '# Last updated: 2023-05-14 23:47:17 (UTC)                      #',
 '#                                                              #',
 '# Terms Of Use: https://urlhaus.abuse.ch/api/                  #',
 '# For questions please contact urlhaus [at] abuse.ch           #',
 '################################################################',
 '#',
 '# url',
 'http://111.61.93.14:37062/i',
 'http://115.55.251.186:45167/bin.sh',
 'http://27.207.1.120:45310/bin.sh',
 'http://117.219.125.152:51355/bin.sh',
 'http://117.235.58.246:53887/bin.sh',
 'http://114.231.242.97:56690/bin.sh',
 'http://182.121.51.197:42384/bin.sh',
 'http://111.61.93.14:37062/bin.sh',
 'http://182.116.107.163:45107/i',
 'http://117.194.171.207:60124/bin.sh',
 'http://178.141.190.97:50387/i',
 'http://117.221.125.129:42611/i',
 'http://123.10.58.253:53832/bin.sh',
 'http://123.14.1

In [24]:

url_list = url_list[9:]
url_list

['http://111.61.93.14:37062/i',
 'http://115.55.251.186:45167/bin.sh',
 'http://27.207.1.120:45310/bin.sh',
 'http://117.219.125.152:51355/bin.sh',
 'http://117.235.58.246:53887/bin.sh',
 'http://114.231.242.97:56690/bin.sh',
 'http://182.121.51.197:42384/bin.sh',
 'http://111.61.93.14:37062/bin.sh',
 'http://182.116.107.163:45107/i',
 'http://117.194.171.207:60124/bin.sh',
 'http://178.141.190.97:50387/i',
 'http://117.221.125.129:42611/i',
 'http://123.10.58.253:53832/bin.sh',
 'http://123.14.114.19:44017/i',
 'http://103.41.27.105:33173/mozi.m',
 'http://42.225.195.201:37602/i',
 'http://61.52.210.98:51634/bin.sh',
 'http://117.221.120.28:39418/i',
 'http://117.253.154.223:55889/i',
 'http://113.118.132.83:56914/i',
 'http://117.195.102.70:52363/i',
 'http://113.86.132.138:44686/i',
 'http://113.86.132.138:44686/bin.sh',
 'http://178.141.190.97:50387/bin.sh',
 'http://117.210.190.7:58672/bin.sh',
 'http://117.253.154.223:55889/bin.sh',
 'http://117.221.125.129:42611/bin.sh',
 'http:

In [98]:
import requests
import zipfile
import io
import os.path
import time

def download_stopforumspam_data():
    url = "https://www.stopforumspam.com/downloads/listed_email_365.zip"
    filename = "listed_email_365.zip"
    extract_file = "listed_email_365.txt"
    
    # check if file was downloaded in last 24 hours
    if os.path.exists(filename) and time.time() - os.path.getmtime(filename) < 86400:
        print("File already downloaded within last 24 hours")
        return
    
    # download zip file
    r = requests.get(url)
    r.raise_for_status()
    
    # extract txt file from zip
    with zipfile.ZipFile(io.BytesIO(r.content)) as zip:
        with zip.open(extract_file) as extract:
            with open(extract_file, "wb") as f:
                f.write(extract.read())
    
    print("StopForumSpam data downloaded and extracted successfully.")


download_stopforumspam_data()

StopForumSpam data downloaded and extracted successfully.


In [25]:
#check if email is in the stopforumspam list
def check_stopforumspam(email):
    with open("listed_email_365.txt", "r") as f:
        for line in f:
            if email in line:
                return True
    return False

# test = check_stopforumspam("opetemi#gmail.com")

# test

In [100]:
url = "http://data.phishtank.com/data/online-valid.json"
response = requests.get(url)

if response.status_code == 200:
    with open("online-valid.json", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download file.")



File downloaded successfully.


In [26]:

#test a URL against the downloaded lists
def test_url(url):
    # check if URL is in URLhaus list
    if url in url_list:
        return "URLhaus"    
    else:
        pass

test_url("signin.ebay.com//aw-cgi/eBayISAPI.dll")



In [27]:
with open('listed_email_365.txt') as f:
    listed_emails = set(f.read().splitlines())

ip_addresses = set()


In [28]:
#check an email address against the downloaded lists
def check_email(email):
    if email in listed_emails:
        return "StopForumSpam"
    return "Unknown"

# test = check_email("sheryl_cochrane73@electricscooter.bar")
# print(test)

In [29]:
def extract_urls(body):
    urls = []
    for url in url_pattern.findall(body):
        urls.append(url)
    return urls


In [30]:
blacklist_words =["Account", "Debit", "Recently","Access", "Information", "Risk", "Bank", "Log", "Security", 
"Client" ,"Notification","Credit", "Online", "Confirm", "Customer", "Password", "Urgent", "Details", "Pay","User"]

In [31]:
class Email:
    def __init__(self,message_id, from_addr, subject, is_multipart, return_path, ips, emails, urls, report):
        self.message_id = message_id
        self.from_addr = from_addr
        self.subject = subject
        self.is_multipart = is_multipart
        self.return_path = return_path
        self.ips = ips
        self.emails = emails
        self.urls = urls
        self.report = report

    def __repr__(self):
        return f"Email(message_id='{self.message_id}', from_addr='{self.from_addr}', subject='{self.subject}', return_path='{self.return_path}', ips={self.ips}, emails={self.emails}, urls={self.urls}, report={self.report})"

In [58]:
email_list = []

def extract_emails(mbox):
    """Extracts email messages from an mbox file and checks if the subject header has words in the blacklist.

    Args:
        mbox: The mbox file to extract emails from.

    Returns:
        A list of email messages.
    """
    for message in mbox:
        message_id = message.get('Message-ID')
        from_addr = message["From"]
        subject = message["Subject"]
        return_path = message["Return-Path"]
        body = message.get_payload()
        is_multipart = message.is_multipart()
        ips = set()
        emails = set()
        urls = set()
        report_set = set()

        if from_addr and return_path and from_addr != return_path:
            report_set.add(f"From ({from_addr}) and Return-Path ({return_path}) for message: {message_id} do not match so it's probably a phishing email.")
        #if from address is == 'None' delete
        elif from_addr == None or from_addr == '' or return_path == None or return_path == '' or subject == None or subject == '':
            continue        
        else:
            pass

        if message_id:
            last_received = None
            for key, value in message.items():
                if key == 'Message-ID':
                    pass
                elif key.startswith('Received') or key.startswith('X-Received'):
                    last_received = value
                    match = ip_pattern.search(value)
                    if match:
                        ips.add(match.group())
                        if match.group() in ip_addresses:
                            report_set.add(f"IP {match.group()} is listed in AbuseIPDB, so it's probably a phishing email.")
                elif key in ('From', 'Return-Path', 'Delivered-To', 'Reply-To'):
                    for email in email_pattern.findall(value):
                        emails.add(email)
                        if email in listed_emails:
                            report_set.add(f"Email {email} is listed in StopForumSpam, so it's probably a phishing email.")
                        else:
                            pass
                elif key == 'Subject':
                    for word in blacklist_words:
                        if word in value:
                            report_set.add(f"Subject contains the word '{word}', so it's probably a phishing email.")
                        else:
                            pass
                elif isinstance(body, str):
                    urls = url_pattern.findall(body)
                    for url in urls:
                        if url in url_list:
                            report_set.add(f"URL {url} is listed in urlhaus, so it's probably a phishing email.")
                        else:
                            pass
                else:
                    pass

        report = list(report_set)
        email = Email(message_id, from_addr, subject, is_multipart, return_path, ips, emails, urls, report)
        email_list.append(email)

extract_emails(mbox)


In [59]:
email_list[1]

Email(message_id='<200502211804.NAA02450@mail.example.com>', from_addr='"emailconfirm@ebay.com" <emailconfirm@ebay.com>', subject='eBay account verification!', return_path='<user@example.com>', ips={'192.168.3.6', '192.168.2.0', '81.196.46.103'}, emails={'emailconfirm@ebay.com', 'user@example.com'}, urls=['http://hform.com', 'http://signin.ebay.com'], report=['From ("emailconfirm@ebay.com" <emailconfirm@ebay.com>) and Return-Path (<user@example.com>) for message: <200502211804.NAA02450@mail.example.com> do not match so it\'s probably a phishing email.'])

In [64]:
#pick 15 random emails
import random
random_emails = random.sample(email_list, 15)

random_emails

[Email(message_id='<JCMAIL01KxQ6kMQGZCU000084b7@jcmail01.jaggedcrossing.com>', from_addr='"Amazon.com"<store-news@amazon.com>', subject='Amazon request: Please follow the Member Verification Procedure!', return_path='<store-news@amazon.com>', ips={'205.245.7.245', '192.168.6.4', '69.15.58.106', '192.168.4.9'}, emails={'store-news@amazon.com', 'user@login.example.com', 'no.reply@amzon.com'}, urls=['http://207.12.161.103', 'http://www.amazon.com', 'http://207.12.161.103', 'http://207.12.161.103', 'http://207.12.161.103'], report=['From ("Amazon.com"<store-news@amazon.com>) and Return-Path (<store-news@amazon.com>) for message: <JCMAIL01KxQ6kMQGZCU000084b7@jcmail01.jaggedcrossing.com> do not match so it\'s probably a phishing email.']),
 Email(message_id='<20060111202131.2DC2F12A684@mail2.tx-rx.org>', from_addr='C U N A <service@cuna.org>', subject='Protect the security of your online account. ', return_path='<user@mail.example.com>', ips={'192.168.3.6', '65.64.220.21', '192.168.6.4'}, em

In [78]:
from datetime import datetime
import uuid
from stix2 import EmailMessage, Indicator, ObservedData, Bundle, EmailAddress, Relationship, CourseOfAction

# create a list to hold the objects for each email
email_objects_list = []

for email in random_emails:
    '''
    Create STIX objects for each email that contains the relevant information
    '''
    
    email_object = EmailMessage(
        type="email-message",
        is_multipart=email.is_multipart,
        from_ref=EmailAddress(type="email-addr", value=email.from_addr),
        sender_ref=EmailAddress(type="email-addr", value=email.return_path),
        message_id=email.message_id,
        subject=email.subject
    )

    # create a list of dictionaries with the required data for creating Indicator objects for this email
    indicators_data = [{'type': 'email-addr', 'value': email_address} for email_address in email.emails]                  

    # create a list to hold Indicator objects for this email
    indicator_objects = []

    # check if the from and return-path email addresses do not match
    if email.from_addr != email.return_path:
        indicator_object = Indicator(
            id=f"indicator--{uuid.uuid4()}",
            pattern_type="Email address headers mismatch",
            pattern = f"[email-message:from_ref.value = '{email.from_addr}'!= email-message:sender_ref.value = '{email.return_path}']",
            created_by_ref="identity--e377678c-67e5-4d8b-91d5-ef0e34588f89",
            labels=["phishing"],
            description = 'from and return paths do not match'
        )
        indicator_objects.append(indicator_object)

    # check if the subject contains words in the blacklist
    for word in blacklist_words:
        if word in email.subject:
            indicator_object = Indicator(
                id=f"indicator--{uuid.uuid4()}",
                pattern_type="Blacklisted word",
                pattern = f"[subject = '{word}']",
                created_by_ref="identity--e377678c-67e5-4d8b-91d5-ef0e34588f89",
                labels=["phishing"],
                description = 'subject header contains phishy keywords'
            )
            indicator_objects.append(indicator_object)

    # create Relationship object to link Indicator to EmailMessage
    
    relationship_object = Relationship(
        relationship_type="indicates",
        source_ref=indicator_object.id,
        target_ref=email_object.id
    )

    # create a list to hold ObservedData objects for this email
    observed_data_objects = []

    observed_data_object = ObservedData(
        id=f"observed-data--{uuid.uuid4()}",
        first_observed=datetime.now(),
        last_observed=datetime.now(),
        number_observed=1,
        created_by_ref="identity--e377678c-67e5-4d8b-91d5-ef0e34588f89",
        object_refs=[email_object.id],
        labels=["email"]
    )

    observed_data_objects.append(observed_data_object)

    course_of_action_object = CourseOfAction(
        id=f"course-of-action--{uuid.uuid4()}",
        name="Send to spam folder",
        description="Move email to spam folder",
        created_by_ref="identity--e377678c-67e5-4d8b-91d5-ef0e34588f89",
        labels=["phishing"]
    )
    # create Relationship object to link CourseOfAction to Indicator
    coa_relationship_object = Relationship(
        relationship_type="mitigates",
        source_ref=course_of_action_object.id,
        target_ref=indicator_object.id
    )

    # concatenate the EmailMessage, Indicator, and ObservedData objects for this email into a single list
    email_objects = [email_object] + indicator_objects + observed_data_objects + [relationship_object] + [course_of_action_object] + [coa_relationship_object]

    # add the list of objects for this email to the list for the entire report
    email_objects_list.append(email_objects)
    
email_objects_list


[[EmailMessage(type='email-message', spec_version='2.1', id='email-message--e14bd43c-d8fc-5c44-9c65-cb63724d0d4e', is_multipart=False, from_ref='email-addr--4d362b9f-6c25-5569-b148-ba3231e94fc3', sender_ref='email-addr--c0d77517-d66b-5940-bb9c-721cf0a9cac2', message_id='<JCMAIL01KxQ6kMQGZCU000084b7@jcmail01.jaggedcrossing.com>', subject='Amazon request: Please follow the Member Verification Procedure!', defanged=False),
  Indicator(type='indicator', spec_version='2.1', id='indicator--c6b1d400-da19-437c-b84c-7ed45245cc7a', created_by_ref='identity--e377678c-67e5-4d8b-91d5-ef0e34588f89', created='2023-05-15T16:27:11.43375Z', modified='2023-05-15T16:27:11.43375Z', description='from and return paths do not match', pattern='[email-message:from_ref.value = \'"Amazon.com"<store-news@amazon.com>\'!= email-message:sender_ref.value = \'<store-news@amazon.com>\']', pattern_type='Email address headers mismatch', valid_from='2023-05-15T16:27:11.43375Z', revoked=False, labels=['phishing']),
  Observ

In [81]:
#create a stix bundle for each item in the email_objects_list
Stix_bundles= []
for email_objects in email_objects_list:
    bundle = Bundle(objects=email_objects)
    Stix_bundles.append(bundle)

#convert list to json
Stix_bundles_json = []
for bundle in Stix_bundles:
    bundle_json = bundle.serialize()
    Stix_bundles_json.append(bundle_json)
    #write json to file
    with open('stix_bundles.json', 'w') as f:
        json.dump(bundle_json, f, indent=4)



In [71]:
import requests
import stix2viz

data = Stix_bundles[0]
stix2viz.display(data)
